## Integrantes:
1. Camila Coltriani
2. Luis Dartayet
3. Irania Fuentes
4. Jonathan Fichelson
5. Ornella Cevoli
# Trabajo práctico  3: Modelos de clasificación 

Introducción: 

El dataset a usar proviene de una empresa de retail en línea cuyo modelo de negocio se sustenta sobre una plataforma que opera únicamente de manera digital. 

El objetivo de este trabajo es identificar clientes que puedan estar propensos a dejar de utilizar la plataforma en el corto plazo.  

In [ ]:
#Las librerías utilizadas en este documento son:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import binarize

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
import seaborn as sns
sns.set()

import statsmodels.api as sm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from collections import ChainMap


In [ ]:
# Booleano para mostrar o no los gráficos del análisis exploratorio de datos
show_EDA_plots = True

## Carga de datos

In [ ]:
data  = pd.read_csv('./data/Datos ML 2021 Q2.csv', sep=';')
print("El dataset tiene {} filas y {} columnas".format(data.shape[0], data.shape[1]))
data.head()

## Descripción del dataset

#### El dataset tiene las siguientes columnas:

- CustomerID: ID del cliente
- Churn: Columna que indica si el cliente dejó de usar la plataforma o no. 1 es que se da de baja.
- CustomerTenure: Es el tiempo transcurrido desde el inicio de la relación con el cliente (en meses)
- MainDeviceLogin: Dispositivo principal que utiliza el cliente para acceder a la plataforma
- CityTier: Indicador del nivel de desarrollo de la ciudad donde vive el cliente
- WarehouseToHome: Distancia desde el centro de distribución a la vivienda del cliente (en km)
- MainPaymentMode: Método de pago más utilizado por el cliente
- Gender: Género del cliente
- HourSpendOnApp: Número de horas que el cliente ha pasado en la plataforma
- DeviceRegistered: Número de dispositivos en los que el cliente ha accedido a la plataforma
- PrefCategory: Categoría más común de las compras del cliente en el último mes
- SatisfactionScore: Nivel de satisfacción del cliente con el servicio
- MaritalStatus: Estado civil del cliente
- NumberOfAddress: Número de direcciones diferentes registradas por el cliente
- Complain: Si ha realizado reclamos
- OrderAmountHikeFromlastYear: Incremento porcentual en la cantidad de compras con respecto al año anterior
- CouponUsed: Número de cupones usados en el último mes
- OrderCount: Número de compras realizadas en el último mes
- DaySinceLastOrder: Cantidad de días desde la última compra
- CashbackAmount: Promedio de reembolsos pedidos en el último mes

Variable objetivo: Churn (termino empleado en marketing para hacer referencia a si un cliente deja de usar una aplicación y/o regresa)

## Exploración de datos

In [ ]:
# Cantidad de datos y chequeo de datos faltantes
data.info()

- Eliminacion de datos NAN

In [ ]:
### Eliminación de valores nan
## Veamos cuantos valores nulos hay en cada columna
data.isna().sum()
## Los eliminamos

In [ ]:
#los eliminamos
data.dropna(inplace=True)

display(data.isna().sum())

print("El dataset limpio tiene {} filas y {} columnas".format(data.shape[0], data.shape[1]))

In [ ]:
# data.describe()

- Distribucion de la variable objetivo *Churn*

In [ ]:
#Distribucion de la variable objetivo
display(data['Churn'].value_counts())
display((data['Churn'].value_counts(normalize=True)*100))

La variable objetivo Churn se distribuye en una clase mayoritaria Churn = 0 (datos = 2802), la cual, corresponde a los clientes que permanecen en la plataforma y una la minoritaria Churn = 1 (datos = 564) que corresponde a los clientes que se dan de baja.

    Observamos que tenemos un dataset desbalanceado con un 83% de datos en la clase 0 y un 16% de datos para la clase 1.

In [ ]:
#Presentamos la distribucion de datos unicos por columna
for columnas in data.columns:
    print("")
    print(f'Nombre:{columnas}')
    print(data[columnas].value_counts())

- Detección de datos sospechosas o atípicos

Se consideran columnas con valores sospechosos aquellas cuya máximo valor se encuentran por encima de 3 desviaciones estándar de la media. 

In [ ]:
#definimos el limite para considerar un dato sospechoso
std_limit = 3
#Por la naturaleza de las variables, se considera que los valores sospechosos son aquellos que se encuentran por encima y no los inferiores.

In [ ]:
# Se crea una funcion para identificar los Columnas con datos sospechosas
suspicious_columns = []

for col in data.columns:
    if(data[col].dtype == 'object'):
        continue
    mean = data[col].mean()
    std = data[col].std()
    max = data[col].max()
    if(max > mean + std_limit*std):
        suspicious_columns.append(data[col].name)
suspicious_columns


In [ ]:
# Se crea una funcion para identificar las filas con datos sospechosas y se grafican
suspicious_rows_arr = []

def investigate_suspicious_column(data, column, watch_outliers=True):
    if(show_EDA_plots):
        fig, ax = plt.subplots(1,2, figsize=(15,5))
        plt.suptitle(column)
        sns.histplot(data[column], ax=ax[0])
        sns.boxplot(data=data[column], ax=ax[1], orient='h')

        plt.show()

    if(watch_outliers):
        mean = data[column].mean()
        std = data[column].std()
        max = data[column].max()

        suspicious_rows = data[data[column] > mean + std_limit*std]
        suspicious_rows_arr.append(suspicious_rows)
        display("Hay {} filas sospechosas".format(suspicious_rows.shape[0]))
        display(suspicious_rows)

In [ ]:
for col in suspicious_columns:
    investigate_suspicious_column(data, col)

In [ ]:
## Total de filas sospechosas
print("Hay {} filas sospechosas".format(sum([suspicious_rows.shape[0] for suspicious_rows in suspicious_rows_arr])))

In [ ]:
## Filas sospechosas agrupadas por columna churn
suspicious_rows = pd.concat(suspicious_rows_arr)
display(suspicious_rows['Churn'].value_counts())
display(suspicious_rows['Churn'].value_counts(normalize=True))


Observamos que la distribución de la variable *Churn* entre los valores extremos es similar, por lo que no parece haber una relación entre las filas con estos datos y la variable objetivo.

Por otro lado, haciendo una observación pormenorizada, creemos que en los casos de las columnas `CouponUsed`, `OrderCount` y `DaySinceLastOrder`  y `CashbackAmount` parecen ser valores lógicos, aún tratándose de valores extremos por lo que no las eliminaremos del dataset original.

Presentamos las columnas que fueron consideradas como sospechosas y las eliminamos de lista: ya no son más columnas sospechosas

In [ ]:
print(suspicious_columns)
suspicious_columns.remove('CouponUsed')
suspicious_columns.remove('OrderCount')
suspicious_columns.remove('DaySinceLastOrder')
suspicious_columns.remove('CashbackAmount')

suspicious_columns

Eliminamos los datos atípicos de las columnas que quedaron como sospechosas:

CustomerTenure, WarehouseToHome, NumberOfAddress

In [ ]:
def remove_outliers(data, column):
    mean = data[column].mean()
    std = data[column].std()
    max = data[column].max()
    return data[data[column] <= mean + std_limit*std]

In [ ]:
for col in suspicious_columns:
    data = remove_outliers(data, col)

Veamos el resultado graficamente

In [ ]:
for col in suspicious_columns:
    investigate_suspicious_column(data, col, False)

### Correlación entre las variables

In [ ]:
#Presentamos los nombres de las 20 columnas
data.columns

- Podemos inferir que las variables que tendrán una significancia en predecir el valor del churn son aquellas en las cuales se puedan observar diferencias en la distribución de las clase de *Churn*: Clase 0 y clase 1.

    Graficamos las variables contra los valores 0 y 1 de *Churn* para verificar esta hipotesis

In [ ]:
if(show_EDA_plots):
    fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(32,32))
    fig.suptitle('Histogramas normalizados')
    for c, ax in zip(data.columns, axes.flatten()):
        sns.histplot(data = data.loc[data['Churn']==0, c].dropna(), stat = 'density', ax = ax, kde = False )
        sns.histplot(data = data.loc[data['Churn']==1, c].dropna(), stat = 'density', kde=False, ax=ax, color = 'orange')
        ax.legend(['Churn = 0', 'Churn = 1'])

De los histogramas podemos observar que: *CustomerID*, *Gender*, *NumberOfAddress* son variables que no permiten diferenciar si el cliente se queda en la plataforma (clase 0) o la abandona (clase 1).

Puede observarse que *CouponUsed* muestra una tendencia a predecir si los datos pertenecen a la clase 0 o clase 1 a medida que aumenta el numero de cupones usados. *OrderCount* por su parte no muestra esta tendencia clara cuando aumenta el numero de compras.


Descartarmos las columnas que no permiten diferenciar las clases en *Churn*

In [ ]:
## Eliminamos las columnas que no permiten diferenciar las clases
columns_to_eliminate = ['CustomerID', 'Gender', 'NumberOfAddress'] ##ojo parece que numberofadress si diferencoa

data = data.drop(columns_to_eliminate, axis=1)

In [ ]:
# if(show_EDA_plots):
#     sns.pairplot(data, hue='Churn')

- Ahora queremos identificar si las variables presentan correlacion entre ellas y con la variable objetivo

In [ ]:
# Vemos que variables están correlacionadas con Churn
if(show_EDA_plots):
    plt.figure(figsize=(20,10))
    sns.heatmap(data.corr(), annot=True, vmin=-1, cmap='Blues')

In [ ]:
#graficamos con pairplot las variables
if(show_EDA_plots):
    sns.pairplot(data)

Interpretacion de la mapa de la correlacion grafica y numerica:

Las variables que muestran correlación (mapa de calor) con la variable objetivo **Churn** son:

- *CustomerTenure* (-0.34), presenta una baja relación, sin embargo, nos indica que es negativa. Es decir, a medida que pasa el tiempo los clientes pueden permanecer en la plataforma o no sin darse de baja. 

- *Complain* (0.24), presenta baja relacion, graficamente nos muestra que los clientes que han realizado más reclamos se han quedado en comparación con los que han dejado la plataforma.

Variables relacionadas entre ellas:

- *OrderCount y CouponUsed* = 0.73, presentan una alta correlación, ya que, es logico que al usar más cupones de compra el numero de ordenes se vea incrementado. 

- *DaySinceLastOrder* y *OrderCount* = 0.45, presentan una buena correlacion, al pasar los dias desde la ultima compra es lógico que las ordenes incrementen. 

- *DaySinceLastOrder* y *CouponUsed* = 0.32, es la misma relacion que con OrderCount

- *CustomerTenure* vs *cashbackAmount* =  0.22, presentan una baja relacion, es logico pensar que si los clientes se quedan más tiempo en la plataforma puede estar asociado a solicitudes de reembolsos.

De esta interpretacion, se puede decidir eliminar como columna predictora *OrderCount* y *DaySinceLastOrder* por estar correlacionas altamente con *CouponUsed*. 

Se decide dejar *CoupondUsed* ya que graficamente (histogramas) se observa que puede diferenciar entre la clase cero y clase 1, mucho mejor que OrderCount.

In [ ]:
## Variables a eliminar
columns_to_eliminate_2 = ['OrderCount', "DaySinceLastOrder"]

data = data.drop(columns_to_eliminate_2, axis=1)

Estadisticamente se considera que una correlacion es despreciable si: r < |0.1|
Vamos a eliminar las variables cuya correlación sea menor a 0.1.

In [ ]:
# ## Correlación entre variables y churn
abs_corr = data.corr()[['Churn']].abs().sort_values(by='Churn', ascending=False)

In [ ]:
## Correlación entre variables y churn, absoluta (para ordenar sin tener en cuenta si es positiva o negativa)
if(show_EDA_plots):
    plt.figure(figsize=(8,12))
    sns.heatmap(abs_corr, annot=True)

In [ ]:
print("Columnas a modelar: ", data.columns)

In [ ]:
#Ordenamos las columnas con datos categoricos
#Para PrefCategory

Nos quedamos con las variables que tienen una correlación mayor a 0.1

In [ ]:
## Variables con correlación mayor a 
high_corr_vars = abs_corr[abs_corr['Churn'] >= 0.1]
## Lo convertimos en una lista para poder iterar sobre ella
high_corr_vars = high_corr_vars.index.tolist()
high_corr_vars

In [ ]:
# # Modificamos nuestro dataset con las columnas de la lista que tienen correlación mayor a 0.1
data = data[high_corr_vars]

*DE AQUI EN ADELANTE NO LO TOQUE*

## Separación de datos

In [ ]:
X = data.drop(['Churn'], axis=1)
y = data['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Verificamos que la proporción de churn sea similar en los conjuntos de entrenamiento y testeo

print("Churn en el conjunto de entrenamiento")
display(y_train.value_counts(normalize=True))
print("Churn en el conjunto de testeo")
display(y_test.value_counts(normalize=True))


## Preparación de datos

Identificamos las columnas categoricas

In [ ]:
categorical_columns = [col for col in data.columns if data[col].dtypes == 'object']

categorical_columns

Identificamos las columnas numericas

In [ ]:
numerical_columns = [col for col in data.columns if data[col].dtypes != 'object']

numerical_columns

In [ ]:
# DeviceRegistered es una variable categórica 

numerical_columns.remove('DeviceRegistered')
categorical_columns.append('DeviceRegistered')

# La transformamos a categórica

X_train['DeviceRegistered'] = X_train['DeviceRegistered'].astype('object')
X_test['DeviceRegistered'] = X_test['DeviceRegistered'].astype('object')


In [ ]:
# Complain es una variable categórica binaria: ha realizado reclamos, no ha realizado reclamos

numerical_columns.remove('Complain')
categorical_columns.append('Complain')

# La transformamos a categórica

X_train['Complain'] = X_train['Complain'].astype('object')
X_test['Complain'] = X_test['Complain'].astype('object')

In [ ]:
display('categorical_columns',categorical_columns)
display('numerical_columns',numerical_columns)

### Variables categóricas: 
Realizamos un OneHotEncoder para convertirlas en numericas

In [ ]:
encoder_categories = []

X_categorical_columns = [x for x in categorical_columns]

for col in X_categorical_columns:    
    col_categories = data[col].unique()
    encoder_categories.append(col_categories)

encoder_categories

In [ ]:
encoder = OneHotEncoder(categories = encoder_categories, sparse=False, drop='first')

encoder = encoder.fit(X_train[X_categorical_columns])

X_train_encoded = encoder.transform(X_train[X_categorical_columns])
X_train_categorical = pd.DataFrame(X_train_encoded, columns = encoder.get_feature_names_out(X_categorical_columns))

X_test_encoded = encoder.transform(X_test[X_categorical_columns])
X_test_categorical = pd.DataFrame(X_test_encoded, columns = encoder.get_feature_names_out(X_categorical_columns))
X_test_categorical.head()

### Variables numéricas

In [ ]:
X_train_numerical = X_train.drop(X_categorical_columns, axis=1)
X_test_numerical = X_test.drop(X_categorical_columns, axis=1)

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_numerical)
X_train_numerical = pd.DataFrame(X_train_scaled, columns = X_train_numerical.columns)

X_test_scaled = scaler.transform(X_test_numerical)
X_test_numerical = pd.DataFrame(X_test_scaled, columns = X_test_numerical.columns)
X_test_numerical.head()

Unimos las variables numéricas y categóricas

In [ ]:
X_train = pd.concat([X_train_categorical, X_train_numerical], axis=1)
X_test = pd.concat([X_test_categorical, X_test_numerical], axis=1)

PENDIENTE- Probar con otro DF, con alguna variable transformada. 

Utilizamos como variables predictoras *CustomerTenure* y *Complain_0*



In [ ]:
X_train = X_train[['CustomerTenure', 'Complain_0']]
X_test = X_test[['CustomerTenure', 'Complain_0']]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train.head()

In [ ]:
# null accuracy - accuracy predicha por un modelo que predice siempre la clase mayoritaria

# total = y_test.shape[0]
# tn = y_test.value_counts()[0]
# fn = y_test.value_counts()[1]
# tp = 0
# fp = 0
# null_accuracy = (tp + tn)/(tp + tn + fp + fn)
# print("TN: {}".format(tn))
# print("FN: {}".format(fn))
# print("Null accuracy: {}".format(null_accuracy))

# También podemos calcularlo con la siguiente función
y_test.value_counts(normalize=True).max()

## Utils para modelos

In [ ]:
def create_confusion_matrixes(y_test, model_pred_dict):
    fix, ax = plt.subplots(1, len(model_pred_dict), figsize=(16.5,4 ))
    for i, model_name in enumerate(model_pred_dict):
        y_pred = model_pred_dict[model_name]['y_pred']
        conf_mat = confusion_matrix(y_test, y_pred)
        conf_mat_df = pd.DataFrame(conf_mat, index = ['Negative (No Churn)', 'Positive (Churn)'], columns = ['Negative (No Churn)', 'Positive (Churn)'])
        sns.heatmap(conf_mat_df, annot=True, fmt='g', cmap='Blues', ax=ax[i])
        ax[i].set_title(model_name)
        ax[i].set_ylabel('True')
        ax[i].set_xlabel('Predicted')
    plt.show()

In [ ]:
def compare_metrics(y_test, model_pred_dict):
    metrics_list = []
    for model_name in model_pred_dict:
        y_pred = model_pred_dict[model_name]['y_pred']
        tn = confusion_matrix(y_test, y_pred)[0,0]
        fp = confusion_matrix(y_test, y_pred)[0,1]
        tp = confusion_matrix(y_test, y_pred)[1,1]
        fn = confusion_matrix(y_test, y_pred)[1,0]
        metrics = pd.DataFrame({'Accuracy': [accuracy_score(y_test, y_pred)], \
                                'Recall': [recall_score(y_test, y_pred)], \
                                'Precision': [precision_score(y_test, y_pred)], \
                                'Specificity': [tn / (tn + fp)], \
                                'F1 score': [f1_score(y_test, y_pred)]})
        metrics.index = [model_name]
        metrics_list.append(metrics)
        

    df = pd.concat(metrics_list, axis=0)
    return df



In [ ]:
def compare_roc_curves(y_test, model_pred_dict):
    for model_name in model_pred_dict:
        y_pred_proba = model_pred_dict[model_name]['y_pred_proba']
        fpr_log,tpr_log,thr_log = roc_curve(y_test, y_pred_proba[:,1])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr_log, tpr_log, label=model_name)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.legend()
    plt.show()

In [ ]:
def compare_auc(y_test, model_pred_dict):
    auc_list = []
    for model_name in model_pred_dict:
        y_pred_proba = model_pred_dict[model_name]['y_pred_proba']
        auc = roc_auc_score(y_test, y_pred_proba[:,1])
        auc_list.append(auc)
    df = pd.DataFrame({'AUC': auc_list}, index=model_pred_dict)
    return df

In [ ]:
def compare_thresholds(y_test, y_pred, y_pred_proba):
    tn = confusion_matrix(y_test, y_pred)[0,0]
    fp = confusion_matrix(y_test, y_pred)[0,1]

    thresholds = np.arange(0, 1, 0.01)
    accuracy = []
    recall = []
    precision = []
    specificity = []
    f1 = []
    for i in thresholds:
        y_pred = binarize(y_pred_proba, threshold=i)[:,1]
        accuracy.append(accuracy_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        precision.append(precision_score(y_test, y_pred, zero_division=0))
        specificity.append(tn / (tn + fp))
        f1.append(f1_score(y_test, y_pred))
    plt.plot(thresholds, accuracy, label='accuracy')
    plt.plot(thresholds, recall, label='recall')
    plt.plot(thresholds, precision, label='precision')
    plt.plot(thresholds, specificity, label='specificity')
    plt.plot(thresholds, f1, label='f1')
    plt.legend()
    plt.xlabel('threshold')
    plt.ylabel('score')
    plt.show()

In [ ]:
def compare_metric_thresholds(y_test, model_pred_dict, method):
    for model_name in model_pred_dict:
        y_pred = model_pred_dict[model_name]['y_pred']
        y_pred_proba = model_pred_dict[model_name]['y_pred_proba']
        tn = confusion_matrix(y_test, y_pred)[0,0]
        fp = confusion_matrix(y_test, y_pred)[0,1]
        thresholds = np.arange(0, 1, 0.01)
        metric = []
        for i in thresholds:
            y_pred = binarize(y_pred_proba, threshold=i)[:,1]
            metric.append(method(y_test, y_pred))
        plt.plot(thresholds, metric, label=model_name)
        plt.legend()
        plt.xlabel('threshold')
        plt.ylabel('score')   

## Diccionario de modelos


In [ ]:
model_pred_dict = {}

def add_to_model_pred_dict(model_name, y_pred, y_pred_proba):
    model_pred_dict[model_name] = {'y_pred': y_pred, 'y_pred_proba': y_pred_proba}

## Naive Bayes

### Modelo

In [ ]:
gnb = GaussianNB()

gnb.fit(X_train, y_train)

In [ ]:
y_pred_nb = gnb.predict(X_test)


In [ ]:
y_pred_proba_nb = gnb.predict_proba(X_test)

In [ ]:
add_to_model_pred_dict('GNB', y_pred_nb, y_pred_proba_nb)

## KNN

### Modelo

In [ ]:
# Utilizamos grid search para encontrar los mejores parámetros

KNN = KNeighborsClassifier()

k_range = list(range(2, 31))
param_grid = dict(n_neighbors=k_range)
print(param_grid)

folds=StratifiedKFold(n_splits=10, shuffle=True)

grid = GridSearchCV(KNN, param_grid, cv=folds, scoring='recall')

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
y_pred_knn = grid.predict(X_test)

In [ ]:
y_pred_proba_knn = grid.predict_proba(X_test)

In [ ]:
add_to_model_pred_dict('KNN', y_pred_knn, y_pred_proba_knn)

## Regresión logística

### Modelo

In [ ]:
# Utilizamos grid search para encontrar los mejores parámetros

lr = LogisticRegression()

c_range = np.logspace(-2, 4, 7)

param_grid = dict(C=c_range)

grid = GridSearchCV(lr, param_grid, cv=folds, scoring='recall')

grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
y_pred_lr = grid.predict(X_test)

In [ ]:
y_pred_proba_lr = grid.predict_proba(X_test)

In [ ]:
add_to_model_pred_dict('LR', y_pred_lr, y_pred_proba_lr)

## Métricas

In [ ]:
create_confusion_matrixes(y_test, model_pred_dict)

In [ ]:
metrics = compare_metrics(y_test, model_pred_dict)

display(metrics)

# Plot metrics

metrics.plot(kind='bar', figsize=(10, 4), xlabel='Model', ylabel='Score', rot=0)



In [ ]:
compare_roc_curves(y_test, model_pred_dict)

In [ ]:
compare_auc(y_test, model_pred_dict)

In [ ]:
compare_metric_thresholds(y_test, model_pred_dict, method=recall_score)

## Conclusiones - PENDIENTE

- Los datos modelados tienen un 85% de casos con clientes que se no abandonan la app y 15 % de clientes que se dan de baja, por lo cual, consideramos que representa un alto sesgo para el modelo.